<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-proyecto-equipo-149/blob/main/Entrega_2_Aguas_subterr%C3%A1neas_(equipo_149).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tecnológico de Monterrey**

# **Maestría en Inteligencia Artificial Aplicada**

# **Curso: Ciencia y analítica de datos**

**Profesora:**
- María de la Paz Rico

**Nombres y matrículas de los integrantes del equipo:**
- Ignacio Valdés Reyes A01793262
- Carlos Gustavo López Lerma A01793468

## Entrega 2

## Instrucciones

Utilizando la base de datos que hayas elegido y hayas realizado su limpieza en la anterior entrega del reto, realiza lo siguiente:

In [46]:
# Importar librerías

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from google.colab import drive

from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import get_scorer
from sklearn.metrics import RocCurveDisplay, precision_recall_curve, roc_curve

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve

In [154]:
# Cargar datos

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datos_de_calidad_del_agua_2020/dfsub2.csv', encoding='latin-1')

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,1,DLAGU6516,POZO R013 CAÃADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,1063,OCRBR5101M1,L-310 (COMUNIDAD SAN MANUEL),RIO BRAVO,NUEVO LEON,LINARES,CITRICOLA SUR,POZO,-99.54191,24.76036,...,NO,SI,NO,SI,SI,SI,SI,SI,SI,SI
1050,1064,OCRBR5102M1,L-305 (EJIDO OJO DE AGUA LAS CRUCESITAS),RIO BRAVO,NUEVO LEON,LINARES,CITRICOLA SUR,POZO,-99.70099,24.78280,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI
1051,1065,OCRBR5105M2,HACIENDA MEXIQUITO POZO 01,RIO BRAVO,NUEVO LEON,CADEREYTA JIMENEZ,CITRICOLA NORTE,POZO,-99.82249,25.55197,...,NO,SI,NO,SI,SI,SI,SI,SI,SI,SI
1052,1066,OCRBR5106M1,COMUNIDAD LOS POCITOS,RIO BRAVO,NUEVO LEON,GALEANA,NAVIDAD-POTOSI-RAICES,POZO,-100.32683,24.80118,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI


## Selecciona tus variables independientes X y dependiente Y (semáforo):

In [155]:
# Ver columnas

df.columns

Index(['Unnamed: 0', 'CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO',
       'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'LONGITUD', 'LATITUD', 'ALC_mg/L',
       'CALIDAD_ALC', 'CONDUCT_mS/cm', 'CALIDAD_CONDUC', 'SDT_M_mg/L',
       'CALIDAD_SDT_ra', 'CALIDAD_SDT_salin', 'FLUORUROS_mg/L', 'CALIDAD_FLUO',
       'DUR_mg/L', 'CALIDAD_DUR', 'COLI_FEC_NMP/100_mL', 'CALIDAD_COLI_FEC',
       'N_NO3_mg/L', 'CALIDAD_N_NO3', 'AS_TOT_mg/L', 'CALIDAD_AS',
       'CD_TOT_mg/L', 'CALIDAD_CD', 'CR_TOT_mg/L', 'CALIDAD_CR', 'HG_TOT_mg/L',
       'CALIDAD_HG', 'PB_TOT_mg/L', 'CALIDAD_PB', 'MN_TOT_mg/L', 'CALIDAD_MN',
       'FE_TOT_mg/L', 'CALIDAD_FE', 'SEMAFORO', 'CONTAMINANTES',
       'CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
       'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
       'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
       'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
       'CUMPLE_CON_FE'],
      dtype='object')

In [156]:
# Categorizar variables por tipos (numéricas, categóricas, binarias y semáforo)

x_num = ['ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L', 'FLUORUROS_mg/L', 'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L', 'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 'HG_TOT_mg/L', 'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L']
x_categ = ['CALIDAD_ALC', 'CALIDAD_CONDUC', 'CALIDAD_SDT_ra', 'CALIDAD_SDT_salin', 'CALIDAD_FLUO', 'CALIDAD_DUR', 'CALIDAD_COLI_FEC', 'CALIDAD_N_NO3', 'CALIDAD_AS', 'CALIDAD_CD', 'CALIDAD_CR', 'CALIDAD_HG', 'CALIDAD_PB', 'CALIDAD_MN', 'CALIDAD_FE']
x_bin = ['CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra', 'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR', 'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD', 'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN', 'CUMPLE_CON_FE']
y = ['SEMAFORO']

In [104]:
x_num

['ALC_mg/L',
 'CONDUCT_mS/cm',
 'SDT_M_mg/L',
 'FLUORUROS_mg/L',
 'DUR_mg/L',
 'COLI_FEC_NMP/100_mL',
 'N_NO3_mg/L',
 'AS_TOT_mg/L',
 'CD_TOT_mg/L',
 'CR_TOT_mg/L',
 'HG_TOT_mg/L',
 'PB_TOT_mg/L',
 'MN_TOT_mg/L',
 'FE_TOT_mg/L']

In [105]:
x_categ

['CALIDAD_ALC',
 'CALIDAD_CONDUC',
 'CALIDAD_SDT_ra',
 'CALIDAD_SDT_salin',
 'CALIDAD_FLUO',
 'CALIDAD_DUR',
 'CALIDAD_COLI_FEC',
 'CALIDAD_N_NO3',
 'CALIDAD_AS',
 'CALIDAD_CD',
 'CALIDAD_CR',
 'CALIDAD_HG',
 'CALIDAD_PB',
 'CALIDAD_MN',
 'CALIDAD_FE']

In [106]:
x_bin

['CUMPLE_CON_ALC',
 'CUMPLE_CON_COND',
 'CUMPLE_CON_SDT_ra',
 'CUMPLE_CON_SDT_salin',
 'CUMPLE_CON_FLUO',
 'CUMPLE_CON_DUR',
 'CUMPLE_CON_CF',
 'CUMPLE_CON_NO3',
 'CUMPLE_CON_AS',
 'CUMPLE_CON_CD',
 'CUMPLE_CON_CR',
 'CUMPLE_CON_HG',
 'CUMPLE_CON_PB',
 'CUMPLE_CON_MN',
 'CUMPLE_CON_FE']

In [107]:
y

['SEMAFORO']

### Selecciona las variables de mayor importancia:

In [157]:
# Eliminar variables sin relevancia para objetivo

df.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)

#df.drop(columns = ['CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'CONTAMINANTES'], axis = 1, inplace = True)

df.drop(columns = ['CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'CONTAMINANTES','CALIDAD_ALC','CALIDAD_CONDUC',
                   'CALIDAD_SDT_ra','CALIDAD_SDT_salin','CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
       'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
       'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
       'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
       'CUMPLE_CON_FE','CALIDAD_FLUO','CALIDAD_DUR','CALIDAD_CR','CALIDAD_HG','CALIDAD_PB','CALIDAD_MN','CALIDAD_FE','CALIDAD_COLI_FEC',
       'CALIDAD_AS','CALIDAD_CD','CALIDAD_N_NO3'], axis = 1, inplace = True)

In [158]:
df.head()

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,FLUORUROS_mg/L,DUR_mg/L,COLI_FEC_NMP/100_mL,N_NO3_mg/L,AS_TOT_mg/L,CD_TOT_mg/L,CR_TOT_mg/L,HG_TOT_mg/L,PB_TOT_mg/L,MN_TOT_mg/L,FE_TOT_mg/L,SEMAFORO
0,-102.02210,22.20887,229.990,940.0,603.6,0.9766,213.7320,1.0,4.184656,0.0161,0.002,0.004,0.0004,0.004,0.0014,0.0891,Verde
1,-102.20075,21.99958,231.990,608.0,445.4,0.9298,185.0514,1.0,5.750110,0.0134,0.002,0.004,0.0004,0.004,0.0014,0.0150,Verde
2,-102.28801,22.36685,204.920,532.0,342.0,1.8045,120.7190,1.0,1.449803,0.0370,0.002,0.004,0.0004,0.004,0.0014,0.0150,Rojo
3,-102.29449,22.18435,327.000,686.0,478.6,1.1229,199.8790,1.0,1.258597,0.0154,0.002,0.005,0.0004,0.004,0.0014,0.0150,Verde
4,-110.24480,23.45138,309.885,1841.0,1179.0,0.2343,476.9872,291.0,15.672251,0.0090,0.002,0.004,0.0004,0.004,0.0014,0.0150,Rojo


In [159]:
df.columns

Index(['LONGITUD', 'LATITUD', 'ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L',
       'FLUORUROS_mg/L', 'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L',
       'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 'HG_TOT_mg/L',
       'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L', 'SEMAFORO'],
      dtype='object')

In [160]:
y = df[['SEMAFORO']]

y

,SEMAFORO
0,Verde
1,Verde
2,Rojo
3,Verde
4,Rojo
...,...
1049,Rojo
1050,Rojo
1051,Rojo
1052,Verde


In [161]:
X = df
X.drop(columns = ['SEMAFORO'],  inplace = True)

X

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,FLUORUROS_mg/L,DUR_mg/L,COLI_FEC_NMP/100_mL,N_NO3_mg/L,AS_TOT_mg/L,CD_TOT_mg/L,CR_TOT_mg/L,HG_TOT_mg/L,PB_TOT_mg/L,MN_TOT_mg/L,FE_TOT_mg/L
0,-102.02210,22.20887,229.990,940.0,603.6000,0.9766,213.7320,1.0,4.184656,0.0161,0.002,0.004,0.0004,0.004,0.00140,0.08910
1,-102.20075,21.99958,231.990,608.0,445.4000,0.9298,185.0514,1.0,5.750110,0.0134,0.002,0.004,0.0004,0.004,0.00140,0.01500
2,-102.28801,22.36685,204.920,532.0,342.0000,1.8045,120.7190,1.0,1.449803,0.0370,0.002,0.004,0.0004,0.004,0.00140,0.01500
3,-102.29449,22.18435,327.000,686.0,478.6000,1.1229,199.8790,1.0,1.258597,0.0154,0.002,0.005,0.0004,0.004,0.00140,0.01500
4,-110.24480,23.45138,309.885,1841.0,1179.0000,0.2343,476.9872,291.0,15.672251,0.0090,0.002,0.004,0.0004,0.004,0.00140,0.01500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,-99.54191,24.76036,231.045,2350.0,1545.8000,0.1900,752.0960,1.0,14.615488,0.0090,0.002,0.004,0.0004,0.004,0.00140,0.01500
1050,-99.70099,24.78280,256.000,529.0,297.0000,0.1900,273.0000,1.0,77.392000,0.0090,0.002,0.004,0.0004,0.004,0.00709,0.07578
1051,-99.82249,25.55197,330.690,2600.0,1873.0000,0.7574,660.2126,620.0,36.477104,0.0090,0.002,0.004,0.0004,0.004,0.02420,0.21290
1052,-100.32683,24.80118,193.140,873.0,690.6667,0.7108,406.3680,1.0,0.010000,0.0090,0.002,0.004,0.0004,0.004,0.01200,0.17860


In [162]:
X.columns

Index(['LONGITUD', 'LATITUD', 'ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L',
       'FLUORUROS_mg/L', 'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L',
       'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 'HG_TOT_mg/L',
       'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L'],
      dtype='object')

## Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]:

In [163]:
# Revisar cantidad de valores por color de semáforo

print(y.value_counts())

SEMAFORO
Verde       427
Rojo        382
Amarillo    245
dtype: int64


In [164]:
# Cambiar etiquetas de semaforo

cambiar_semaforo = {'Verde' : 1, 'Amarillo' : 2, 'Rojo' : 3}
y['SEMAFORO'] = y['SEMAFORO'].replace(cambiar_semaforo)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [165]:
y

,SEMAFORO
0,1
1,1
2,3
3,1
4,3
...,...
1049,3
1050,3
1051,3
1052,1


## Realiza un análisis general de las features importances a traves de decision trees o random forest y explora que clasificador es el más optimo, ejemplo:

- Decision trees
- Random Forest

In [166]:
Xtv, Xtest, ytv, ytest = train_test_split(X, y, test_size = 0.20, random_state = 7)

In [167]:
print('Dimensión del conjunto de entrenamiento variables independientes: ', Xtv.shape)
print('Dimensión del conjunto de prueba variables independientes: ', Xtest.shape, '\n')
print('Dimensión del conjunto de entrenamiento variable dependiente: ', ytv.shape)
print('Dimensión del conjunto de prueba variable dependiente: ', ytest.shape)

Dimensión del conjunto de entrenamiento variables independientes:  (843, 16)
Dimensión del conjunto de prueba variables independientes:  (211, 16) 

Dimensión del conjunto de entrenamiento variable dependiente:  (843, 1)
Dimensión del conjunto de prueba variable dependiente:  (211, 1)


In [170]:
# Random Forest Classifier
forest = RandomForestClassifier(n_estimators = 100, random_state = 0)
forest.fit(Xtv, np.ravel(ytv))
prioridad = pd.DataFrame(index = Xtv.columns, data = forest.feature_importances_, columns = ["Prioridad"])
prioridad.sort_values(by = "Prioridad", ascending = False).head(5)


,Prioridad
FLUORUROS_mg/L,0.208240
DUR_mg/L,0.134547
AS_TOT_mg/L,0.108720
N_NO3_mg/L,0.089669
MN_TOT_mg/L,0.079319


Del análisis de Random Forest podemos concluir que la variable más importante es FLUORUROS_mg/L

## Explora que clasificador es el más optimo, ejemplo:

- Decision trees
- Random Forest

## Determina el grado de exactitud a través del reporte de clasificación:

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

## Visualiza los resultados del modelo o las predicciones a través de una matriz de confusión:

## Realiza un reporte de los principales hallazgos y conclusiones del experimento e incluyelos en una presentación ejecutiva de 10 diapositivas. La presentación deberá incluir todos los pasos del pipeline seguidos, limpieza, análisis, kmeans, clasificación, resultados y conclusiones: